In [3]:

# Fatemeh Makhloughi
import os 
from PIL import Image
directory_path = r'F:/Research/dataset/Final_1Hz/All_images'

max_subject = 0
max_epochs = {}

for filename in os.listdir(directory_path):
    if filename.startswith('Subject_'):
        parts = filename.split('_')
        if len(parts) == 6:
            subject = int(parts[1])
            epoch = int(parts[5].split('.')[0]) 
            max_subject = max(max_subject, subject)
            if subject not in max_epochs or epoch > max_epochs[subject]:
                max_epochs[subject] = epoch
                
for subject in range(1,66):
    subject_dir = f'F:/Research/dataset/Final_1Hz_con/Subject_{subject}/'
    os.makedirs(subject_dir, exist_ok=True)

    max_epoch = max_epochs.get(subject, 0)  

    for epoch in range(1,max_epoch+1):  
        image_paths = []

        for channel in range(1,20):
            filename = f'Subject_{subject}_Channel_{channel}_Epoch_{epoch}.jpg'
            file_path = os.path.join(directory_path, filename)

            if os.path.exists(file_path):
                image_paths.append(file_path)
            else:
                print(f"File {filename} not found.")

        loaded_images = []

        for path in image_paths:
            img = Image.open(path)
            img = img.crop((90, 50, img.width - 150, img.height - 60))
            loaded_images.append(img)

        pattern = [
            [None, 0, None, 1, None],
            [10, 2, 16, 3, 11],
            [12, 4, 17, 5, 13],
            [14, 6, 18, 7, 15],
            [None, 8, None, 9, None]
        ]

        concatenated_image = Image.new('RGB', (5 * loaded_images[0].width, 5 * loaded_images[0].height))

        coordinates = [(j * loaded_images[0].width, i * loaded_images[0].height) for i, row in enumerate(pattern) for j, col in enumerate(row)]
        for i, coord in enumerate(coordinates):
            img = loaded_images[pattern[i // 5][i % 5]] if pattern[i // 5][i % 5] is not None else Image.new('RGB', (loaded_images[0].width, loaded_images[0].height), (255, 255, 255))
            concatenated_image.paste(img, coord)

        concatenated_image.save(os.path.join(subject_dir, f'Subject_{subject}_Epoch_{epoch}_concatenated.jpg'))
